In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
zip_path = "/content/shapes.zip"

In [ ]:
#unzip the rar file 
import zipfile
import os
!apt-get install -y unrar > /dev/null
!pip install -q patool

In [ ]:
import patoolib
import os

In [ ]:
rar_path = "/content/shapes.zip"
extract_path = "/content/shapes"

os.makedirs(extract_path, exist_ok=True)

In [ ]:
# Extract the .rar file
patoolib.extract_archive(rar_path, outdir=extract_path)

print("✅ Extraction complete!")

In [ ]:
for root, dirs, files in os.walk(extract_path):
    level = root.replace(extract_path, '').count(os.sep)
    indent = ' ' * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 4 * (level + 1)
    for f in files:
        print(f"{subindent}{f}")

In [ ]:

import shutil
import random

# Original dataset path
source_dir = "/content/shapes/shapes"

# New directory for split data
base_dir = "/content/shapes_split"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")

# Create directories
for dir_path in [train_dir, test_dir]:
    os.makedirs(dir_path, exist_ok=True)

# Split ratio
split_ratio = 0.8  # 80% train, 20% test

# Loop through each class folder
for class_name in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_name)
    if os.path.isdir(class_path):
        images = os.listdir(class_path)
        random.shuffle(images)

        split_index = int(len(images) * split_ratio)
        train_images = images[:split_index]
        test_images = images[split_index:]

        # Create subfolders in train/test
        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

        # Move images
        for img in train_images:
            shutil.copy(os.path.join(class_path, img),
                        os.path.join(train_dir, class_name, img))

        for img in test_images:
            shutil.copy(os.path.join(class_path, img),
                        os.path.join(test_dir, class_name, img))

print("✅ Data split completed!")
print(f"Train folder: {train_dir}")
print(f"Test folder: {test_dir}")


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

In [ ]:
train_dir = "/content/shapes_split/train"
test_dir = "/content/shapes_split/test"


In [ ]:
# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Image size and batch size
img_size = (128, 128)
batch_size = 32

# Create data generators
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)


In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')   # 4 classes
])

model.summary()


In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
print(train_gen.class_indices)


In [ ]:
history = model.fit(
    train_gen,
    validation_data=test_gen,
    epochs=15
)
#this is the training of the model

In [ ]:
plt.figure(figsize=(10,4))

# Accuracy
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.legend()

# Loss
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.legend()

plt.show()


In [ ]:
test_loss, test_acc = model.evaluate(test_gen)
print(f"✅ Test Accuracy: {test_acc*100:.2f}%")


In [ ]:
model.save("/content/shape_classifier.h5")
print("✅ Model saved as shape_classifier.h5")


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = "/content/shapes_split/test/circle/1001.png"  # change path
img = image.load_img(img_path, target_size=(128, 128))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)
class_labels = list(train_gen.class_indices.keys())

print("Predicted Shape:", class_labels[np.argmax(prediction)])


In [ ]:
img_path = "/content/images.jpg"  # change path
img = image.load_img(img_path, target_size=(128, 128))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)
class_labels = list(train_gen.class_indices.keys())

print("Predicted Shape:", class_labels[np.argmax(prediction)])
